## 🤖 Agentic Chatbot with LangGraph

An agentic chatbot leverages KangGraph to build complex, autonomous, and dynamic conversational flows. KangGraph, a framework within LangChain, enables the creation of state machines that manage the chatbot's behavior, allowing it to make decisions, use tools, and respond to user input in a more sophisticated way than traditional chatbots.

#### Here's a more detailed explanation:

**Traditional Chatbots:** Typically follow pre-defined rules and scripts, offering limited conversational flexibility.

**Agentic Chatbots:**
Employ AI agents with reasoning and decision-making capabilities. They can adapt their responses, use external tools, and handle complex tasks, says Salesforce.

**KangGraph's Role:**
KangGraph is the core framework that enables the construction of agentic chatbots. It allows developers to define the chatbot's state transitions, routing logic, and tool usage, says Medium.

Example: \\
Imagine a weather chatbot. A traditional chatbot might simply provide the current weather. An agentic chatbot, using KangGraph, could also:
- Ask if the user wants the forecast for tomorrow, says Medium
- Check the user's location (with user consent) to provide a more relevant forecast
- Alert the user if there's a severe weather warning in their area, says Medium.

according to [Medium](https://medium.com/data-science-collective/langgraph-mcp-ollama-the-key-to-powerful-agentic-ai-e1881f43cf63).

#### Benefits of Agentic Chatbots:
- **Higher Personalization:** Tailor responses based on user profiles and preferences.
- **Complex Task Handling:** Manage intricate inquiries and tasks beyond the capabilities of traditional chatbots.
- **Autonomous Decision-Making:** Agents can make decisions and take actions without constant human supervision.

#### What is ReAct and how is it related to LangGraph and Agentic AI?

**ReAct (Reasoning and Acting)** is a powerful pattern that combines reasoning traces with actions. ReAct architecture basically refers to a framework that combines reasoning and action to enable AI agents to solve complex tasks and decision-making in agentic workflows. It's a machine learning paradigm that integrates the reasoning and action-taking capabilities of large language models (LLMs). ReAct agents don't separate decision-making from task execution, unlike traditional AI systems.

In LangGraph, we can implement this by adding a reasoning step before each action.



The agentic chatbot implemented in this specific project is designed to autonomously process user queries by dynamically deciding the best (or even all) sources of information, utilizing a LangGraph-based flow. It leverages specialized runners and API wrappers to search and retrieve data from Wikipedia for general knowledge and from Arxiv for academic research. Based on the nature of the query, the chatbot intelligently orchestrates these resources in parallel, aggregates the retrieved information, and synthesizes a clear, coherent final response. Its modular design ensures flexible, efficient information gathering and enables the chatbot to handle both casual and technical inquiries with depth and precision.

In [ ]:
# Install the necessary libraries and frameworks
!pip install -q -U pydantic langchain langgraph langchain-core langchain-community
!pip install -q -U python-dotenv langchain-groq
!pip install -q -U arxiv
!pip install -q -U wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.2/437.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.8 MB/s eta 0:00:00
  Prepari

What do you think enables us to utilize resources like Wikipedia and Arxiv to gather all the information we need? \\
You're right! \\
Simply through the two pairs of tools listed below to deal with Wikipedia and Arxiv content:
1. `WikipediaAPIWrapper`, `WikipediaQueryRun`
2. `ArxivAPIWrapper`, `ArxivQueryRun`

**ArxivAPIWrapper** and **WikipediaAPIWrapper**:

These are lower-level helpers or tools which wrap the raw API (Arxiv API, Wikipedia API) and provide easy Python methods to interact with them (like `.run()`, `.search()`). *They don't decide when/how to query — they just provide functions for querying.* Think of them as "tools" that the `QueryRun` classes use underneath.

**ArxivQueryRun** and **WikipediaQueryRun**:

These are Runnable classes or functions in LangChain (or similar frameworks) designed to execute a search/query operation directly. They take a query (like a research topic or a question) and return search results (papers from Arxiv, articles from Wikipedia). Think of them as the "doers" — you give them a task, and they go fetch the data.

#### In short:

|                | **QueryRun** (Runner)             | **APIWrapper** (Helper)             |
|----------------|------------------------------------|-------------------------------------|
| Purpose        | Run full queries easily            | Provide methods to query APIs       |
| Usage          | High-level, one-step search        | Lower-level, more flexible          |
| Think of as    | "Do it for me"                     | "Give me tools, I'll do it"          |
| Example        | `.invoke("machine learning")`      | `.run("machine learning")`           |


#### ⚡ A Real-world Analogy:

APIWrapper = You own a car (manual control — steering wheel, brakes).

QueryRun = You hire a driver (just say "take me to the library," and he drives).

In [ ]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [ ]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper, description="Used to query Arxiv for recent papers.")

In [ ]:
print(arxiv.name)

arxiv


In [ ]:
# Make query to retrieve papers on a specific topic
arxiv.invoke("Logic Tensor Network")

"Published: 2016-07-07\nTitle: Logic Tensor Networks: Deep Learning and Logical Reasoning from Data and Knowledge\nAuthors: Luciano Serafini, Artur d'Avila Garcez\nSummary: We propose Logic Tensor Networks: a uniform framework for integrating\nautomatic learning and reasoning. A logic formalism called Real Logic is\ndefined on a first-order language whereby formulas have truth-value in the\ninterval [0,1] and semantics defined concretely on the domain of real numbers.\nLogical constants are interpreted "

In [ ]:
wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wikipedia = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper, description="Used to query Wikipedia for recent articles.")
wikipedia.name

'wikipedia'

In [ ]:
# Make query to retrieve articles on a specific topic
# Note that the number of content characters is already set andlimited to 500
wikipedia.invoke("LangGraph")

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis."

Very well!  \\
Having prepared the tools above regarding particular resources (Wikipedia & Arxiv in our case) through which one can perform real-time information extraction, we can also consider a search engine API designed specifically for AI agents and Large Language Models (LLMs), for instance, **Tavily** to search the live web for broad, up-to-date information.

Tavily is a search engine API designed specifically for AI agents and Large Language Models (LLMs). It provides real-time, accurate, and relevant information from the web, tailored for AI applications like those using Retrieval-Augmented Generation (RAG). Unlike traditional search APIs, *Tavily focuses on delivering information in a format optimized for AI processing, including dynamically searching the web, reviewing multiple sources, and extracting relevant content.*

In [ ]:
# Get api keys for both Groq and Tavily

from google.colab import userdata
import os

os.environ["TAVILY_API_KEY"] = userdata.get("tavily-api-key")
os.environ["GROQ_API_KEY"] = userdata.get("groq-api-key")

In [ ]:
# Set up Tavily search tool

from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults()

In [ ]:
tavily.invoke("Provide news on the most modern techs.")

[{'title': 'TechNewsWorld - Technology News and Information',
  'url': 'https://www.technewsworld.com/',
  'content': "Audio/Video\nEmerging Tech\nArtificial Intelligence\nRobotics\nVirtual Reality\n\n\nGaming\nHome Tech\nHow To\nPhotography\nScience\nHealth\nSpace\n\n\nTech Buzz\nTech Law\nTransportation\nWomen In Tech\n\nNewsletters\nSee all Newsletters\n\nE-Commerce Minute\nTech News Flash\n\nEditors' Pick\n\n\nCRM Buyer\n\nE-Commerce Times\n\nLinuxInsider\n\n\nTechNewsWorld\n\nTop Stories [...] Online Entertainment\nSearch Tech\nSocial Networking\nWeb Apps\n\nIT\n\n\nDevelopers\n\nIT Leadership\nNetwork Management\n\nMobile Tech\n\n\nMobile Apps\n\nSmartphones\nTablets\nWearable Tech\nWireless Networking\n\nReviews\n\n\n\n\nSecurity\n\nCybersecurity\nHacking\nMalware\nPrivacy\n\nTech Blog\n\n\n\n\nTechnology\n\nAudio/Video\nEmerging Tech\nArtificial Intelligence\nRobotics\nVirtual Reality\n\n\nGaming\nHome Tech\nHow To\nPhotography\nScience\nHealth\nSpace\n\n\nTech Buzz\nTech Law\n

Putting all tools together, you can exploit an LLM model via Grok Cloud to integrate with...

In [ ]:
# embrace all the tools in a list
tools = [arxiv, wikipedia, tavily]

In [ ]:
# initialize the LLM model
from langchain_groq import ChatGroq

llm=ChatGroq(model="qwen-qwq-32b")

In [ ]:
llm.invoke("What is Deep Learning?")

AIMessage(content='\n<think>\nOkay, so I need to explain what deep learning is. Let me start by recalling what I know. Deep learning is part of machine learning, right? It\'s related to neural networks, especially the ones with multiple layers. But wait, how exactly does it differ from regular neural networks? Maybe it\'s when the networks are "deep," meaning they have many layers, hence the name. But I should make sure that\'s accurate.\n\nI remember that machine learning is about algorithms learning from data, and deep learning is a subset that uses these multi-layered neural networks. These networks are inspired by the human brain\'s structure, with neurons and synapses. The layers in a deep learning network are called hidden layers, and each layer processes information in a way that builds on the previous layer. The inputs go through these layers, and each layer extracts features from the data. For example, in image recognition, the first layers might detect edges, then more comple

In [ ]:
# bind llm with tools
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
results = llm_with_tools.invoke("What isnew on AI?")

In [ ]:
results

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gy9j', 'function': {'arguments': '{"query": "latest advances in artificial intelligence"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_vjhk', 'function': {'arguments': '{"query": "recent advancements in AI technology 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 315, 'prompt_tokens': 329, 'total_tokens': 644, 'completion_time': 0.765895693, 'prompt_time': 0.025285404, 'queue_time': 0.179575552, 'total_time': 0.791181097}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_07cd5d759a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-485e829d-6efc-4bb9-a1f4-411a3f5e827f-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'latest advances in artificial intelligence'}, 'id': 'call_gy9j', 'type': 'tool_call'}, {'name': 'tavily_search_results_json', 'args': {'query': 'recent advancements in AI technology 2023'}, 'id': 'c

''